# Tutorial 1: using a Quantum Device to solve MIS


Maximum Independent Set (MIS) is a standard and widespread graph problem in scheduling, network theory, error correction, and even in the quantum sector as part of more general optimization algorithms (e.g., QUBO formulations) or as a benchmark on quantum annealers or neutral atom devices.

However, there is no known polynomial-time algorithm for general graphs and even approximating it is hard in some cases.

The high-level goal of this tutorial is to execute an instance of the classic $NP$-hard MIS problem, using both classical and quantum methods. 

The MIS problem can be formulated as follows: given a graph $G=(V,E)$, an independent set is a subset of vertices $S\subseteq V$ such that no two vertices in $S$ are connected by an edge. The MIS problem then seeks to find the largest independent set in $G$.

By the end of this notebook, you will know how to:

- Setup import for standard MIS benchmarking DIMACS datasets.
- Setup compilation and execution of these graphs for execution on both Classical and Quantum Device (either an emulator or a physical QPU).
- Launch the execution and extract relevant results.

## Dataset preparation

As in any MIS process, we first need to load and prepare data in a suitable graph format. For this tutorial, we will use the standard benchmark [DIMACS datasets](https://oeis.org/A265032/a265032.html) of various sizes and convert them to supported [Networkx](https://networkx.org/documentation/stable/index.html#) graph types.

In [ ]:
# Ignore warnings for this tutorial.
import logging
import os
import sys

logger = logging.getLogger()
logger.disabled = True

sys.stderr = open(os.devnull, 'w')

In [ ]:
import networkx as nx

# Create a new networkx graph instance to be populated with DIMACS data.
graph = nx.Graph()


with open("./datasets/dimacs/a265032_1tc.32.txt", "r") as f:
    for line in f:
        if line.startswith("c"):  # Comment line in DIMACS file.
            continue
        elif line.startswith("p"):  # Problem definition, i.e. # nodes and edges.
            _, _, num_nodes, num_edges = line.strip().split()
            # Preset graph node labels as there might be isolated ones.
            graph.add_nodes_from(range(1, int(num_nodes) + 1))

        elif line.startswith("e"):
            _, node1, node2 = line.strip().split()
            graph.add_edge(int(node1), int(node2))

# Let's check what the graph looks like.
print(graph)


## Solving the MIS using Networkx's heuristic solver

Let's first solve this instance of MIS using the standard classical and heuristic [MIS solver in Networkx](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.approximation.clique.maximum_independent_set.html). From its inherent heuristic and non-deterministic nature, this solver does not guarantee optimality in solution finding.

In [ ]:
from mis.solver.solver import MISInstance
from mis.pipeline.config import SolverConfig
from mis import MISSolver
# Define classical solver configuration
from mis.shared.types import MethodType

# Configure the library to use a classical (non-quantum) heuristic solver.
config = SolverConfig(
    method = MethodType.EAGER,
    max_iterations=1
)

# Create the MIS instance
instance = MISInstance(graph)

# Run the solver and retrieve results.
solver = MISSolver(instance, config)
solutions = solver.solve().result()

# Display results
print("MIS solution:", solutions[0].nodes)
print("Solution cost:", solutions[0].frequency)

The solver returns a list of node labels of length 12 which is the unique solution so far (frequency of 1.).

## Solving using the quantum SDK QuTiP

In the previous section, we have used a non-quantum solver to resolve our instance of MIS. In this section, we'll use actually use a quantum algorithm. There are three steps to quantum algorithms:
1. Converting the problem into a Register (the position of atoms in the quantum device) and a set of Pulses (the configuration of lasers on the quantum device)
2. Actually running the Register and Pulse on the quantum device
3. Extracting the results of quantum measurement on the quantum device into a solution to the original problem.

In this library, all three steps are entrusted to _backends_. This library provides several backends, depending on your use. Since you may not have access to a quantum computer for step 2, we will use the `QutipBackend`. This is a simple backend that has the advantage of working on most computers, regardless of operating system or GPU.

In [ ]:
from mis.pipeline.backends import QutipBackend


config = SolverConfig(
    method = MethodType.EAGER,
    backend = QutipBackend(),
    max_iterations=1
)

# Create the MIS instance
instance = MISInstance(graph)


# Run the solver
solver = MISSolver(instance, config)
solutions = solver.solve().result()

# Display results
print("MIS solution:", solutions[0].nodes)
print("Solution cost:", solutions[0].frequency)

We can see that results are coherent with the solution from the classical solver above.

## Solving using Remote QPU backend

This section illustrates the use of a QPU backend hosted on Pasqal's Cloud Platform. Provided that you are granted with credentials to access the platform, they should be passed to instantiate a `RemoteQPUBackend`.

In [ ]:
from mis.pipeline.backends import RemoteQPUBackend

USERNAME="username"
PROJECT_ID="123"
PASSWORD=None

if PASSWORD is not None:
    config = SolverConfig(
        method = MethodType.EAGER,
        backend = RemoteQPUBackend(
            username=USERNAME,
            project_id=PROJECT_ID,
            password=PASSWORD
        ),
        max_iterations=1
    )

    # Create the MIS instance
    instance = MISInstance(graph)

    # Run the solver
    solver = MISSolver(instance, config)
    solutions = solver.solve().result()

    # Display results
    print("MIS solution:", solutions[0].nodes)
    print("Solution cost:", solutions[0].frequency)